# Utils

> Various useful pieces

In [ ]:
#| default_exp utils

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()
from nbdev.showdoc import *

In [ ]:
#| export
from pydantic_settings import BaseSettings
from pathlib import Path
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

False

In [ ]:
#| export

class Source(BaseSettings):
    name: str
    doc_dir: Path


class GitSource(Source):
    repo: str
    commit: str

class WebSource(Source):
    pass


class Settings(BaseSettings):
    model: str = "claude-haiku-4.5"
    git_dir: Path = Path("../git_dir") # Cloned git repos will be here
    output_dir: Path = Path("../processed_documents/")
    templates_dir: Path = Path("../templates")
    project_root: Path = Path(__file__).parent.parent if "__file__" in globals() else Path.cwd()
    sources: dict[str, Source] = {
        "sveltejs/svelte": GitSource(
            name="sveltejs-svelte",
            doc_dir="documentation/docs",
            repo="https://github.com/sveltejs/svelte",
            commit="main")
    }
    api_key: str = os.getenv("ANTHROPIC_API_KEY", "")


settings = Settings()

In [ ]:
#| export

from git import Repo, InvalidGitRepositoryError, NoSuchPathError

### Working with git

In [ ]:
#| export

def git_progress(op_code, cur_count, max_count=None, message=''):
    if max_count:
        print(f"\r{op_code}: {cur_count}/{max_count} {message}", end='', flush=True)
    else:
        print(f"\r{op_code}: {cur_count} {message}", end='', flush=True)


def clone_repo(source: GitSource):
    """Clone a git repository if it doesn't exist, or ensure an existing one is clean and at the specified commit.

    Args:
        source (GitSource): Configuration object containing the repository URL, name, and target commit.

    Returns:
        None

    Will raise exceptions on errors
    """
    clone_dir = settings.git_dir / source.name

    try:
        repo = Repo(clone_dir)
        repo.git.clean('-fdq') # Just in case
    except (InvalidGitRepositoryError, NoSuchPathError):
        repo = Repo.clone_from(source.repo, clone_dir, progress=git_progress)

    repo.git.reset('--hard', source.commit)